# Kết nối GDrive

In [ ]:
from google.colab import drive

drive.mount("/content/GDrive")

Mounted at /content/GDrive


# Import thư viện

In [ ]:
import pandas as pd
import numpy as np
import sys
import glob
import os
from sklearn.feature_extraction.text import * 
from nltk.tokenize import word_tokenize
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math
import operator
import string
import nltk
from collections import Counter,defaultdict
from nltk.stem import WordNetLemmatizer,PorterStemmer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
stop_word = stopwords.words('english')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Tiền xử lý data

In [ ]:
def preprocess_data(data):
  # chữ cái thường
  data = data.lower()
  # loại bỏ punctuation và ký tự đặc biệt
  data = data.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
  data = data.translate(str.maketrans("‘’’–——−", '       '))
  data = re.sub('\n',' ',data)
  data = re.sub('[^A-Za-z]',' ',data)
  return data

In [ ]:
import os
text = []
path_fold = '/content/GDrive/MyDrive/test_small'
listfName = os.listdir(path_fold)

for i in range(len(listfName)):
  f = open(os.path.join(path_fold, listfName[i]), 'r', encoding='UTF-8')
  data = f.read()
  data = preprocess_data(data)
  text.append(data)
f.close()

In [ ]:

def split_word(data):
  return word_tokenize(data)
def remove_stopwords(data):
  list_filter = []
  for word in data:
    if word not in stop_word:
      list_filter.append(word)
  return list_filter
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
def lemma_stem(action,data):
  if action == 1:
      stemmed = []
      for i in data:
        stemmed.append(stemmer.stem(i))
      return stemmed
  elif action == 0:
      lemmatized = []
      for i in range(0,len(data)):
        lemmatized.append(lemmatizer.lemmatize(data[i]))
      return lemmatized
  else:
    list_w = []
    for i in range(0,len(data)):
        list_w.append(data[i])
    return list_w
def process(data,action):
  list_words = []
  for i in range(0,len(data)):
    text = split_word(data[i])
    text = remove_stopwords(text) 
    result = []
    if action == 1:
      for i in text:
          result.append(stemmer.stem(i))
    else:
      for i in text:
          result.append(stemmer.stem(i))
    list_words.append(result)
  return list_words

In [ ]:
list_words = process(text,0)

# Lập chỉ mục đảo ngược

In [ ]:
start_time = time.time()
data = []
fred = dict()
for id_doc in range(0,len(list_words)):
  for j in Counter(list_words[id_doc]):
      counter =[]
      counter.append(j)
      counter.append((id_doc+1,Counter(list_words[id_doc])[j]))
      counter.append(Counter(list_words[id_doc])[j])
      if j not in fred:
        fred[j] = Counter(list_words[id_doc])[j]
      else:
        fred[j] += Counter(list_words[id_doc])[j]
      data.append(counter)
unique_words = defaultdict(list)
for line in data:
  unique_words[line[0]].append(line[1])
List = []
for item in unique_words:
   line = []
   line.append(item)
   line.append(unique_words[item])
   line.append(fred[item])
   line.append(len(unique_words[item]))
   List.append(line)
end_time = time.time()

# Tạo dictionary tần số term trong 1 document

In [ ]:
def termFrequencyInDoc(List):
    termFreqADoc=dict()
    for line in List:    
      term = line[0] 
      for id_doc,count in line[1]:
        if id_doc not in termFreqADoc:
            termFreqADoc[id_doc] = {}
            termFreqADoc[id_doc][term] = count
        else:
           termFreqADoc[id_doc][term] = count
    return termFreqADoc

In [ ]:
termFreqADoc = termFrequencyInDoc(List)
termFreqADoc

{1: {'two': 1, 'squar': 1, 'root': 1, 'approxim': 1},
 2: {'root': 1,
  'extract': 1,
  'repeat': 1,
  'subtract': 1,
  'digit': 1,
  'comput': 1},
 3: {'techniqu': 1, 'depart': 1, 'matrix': 1, 'program': 1, 'scheme': 1},
 4: {'preliminari': 1, 'report': 1, 'intern': 1, 'algebra': 1, 'languag': 1}}

# GVSM

In [ ]:
minterm =[]
for doc in termFreqADoc:
  cn=0
  val=0
  for value in termFreqADoc[doc]:
    if termFreqADoc[doc][value]>0:
      val=val+pow(2,cn)
    cn=cn+1
  minterm=minterm+[val]		

unit_vectors=[]
tot_words=len(List)

In [ ]:
minterm

[15, 63, 31, 31]

document 1 has minterm(15),document 2 has minterm(63),document 3 has minterm(31),document 4 has minterm(31)

In [ ]:
p=[]
size_of_minterms=pow(2,(tot_words)) #size of GVSM vector space is 2^total_words
tmp = [0 for j in range(pow(2,tot_words))]

for i in range(0,tot_words):
	tmp_unit_vector = tmp
	cnt=0
	for doc in termFreqADoc:
		cn=0
		for value in termFreqADoc[doc]:
			if i == cn:
				tmp_unit_vector[minterm[cnt]]=tmp_unit_vector[minterm[cnt]]+termFreqADoc[doc][value]
				#print(tmp_unit_vector[minterm[cnt]])
			cn=cn+1	
		cnt=cnt+1

	magnitude=np.linalg.norm(tmp_unit_vector)
	myArr=np.array(tmp_unit_vector)
	newArr=myArr/magnitude
	p.append(newArr)


Em lấy Document 1 làm câu truy vấn, document 2-4 làm tập tài liệu cần tính xếp hạng

In [ ]:
from numpy.linalg import norm

queryVector=np.zeros(pow(2,tot_words))
sim=[]
count_file=0

#this loop constructs document and query vectors in the GVSM vector space as linear combination of minterm vectors and cosine similarity
for doc in termFreqADoc:

	docVector=np.zeros(pow(2,tot_words))
	count=0 #keeps count of index terms

	for value in termFreqADoc[doc]:
		docVector+=(termFreqADoc[doc][value]*p[count]) #product of tfidf value from matrix and nomrmalised term vectors from p
		count=count+1
	B=np.array(docVector)
	if(count_file==0):
		queryVector+=docVector #assign query vector
		A=np.array(queryVector)
	sim.append(np.dot(A,B)/(norm(A)*norm(B))) #cosine similarity

	count_file+=1

#sort the documents based on their similarity from highest to lowest similarity order
sim.sort(reverse=True)

count_file=0
for i in range(1,4):
   print('Doc ',i,': ',text[i])
print('Query: ',text[0])
print('Ranking by cosine similarity:')
#print the documents' ranking
for s in sim[1:]:
	print('docId', count_file+1,' : ',s)
	count_file+=1

Doc  1 :  extraction of roots by repeated subtractions for digital computers 
Doc  2 :  techniques department on matrix program schemes 
Doc  3 :  preliminary report international algebraic language 
Query:  two square root approximations 
Ranking by cosine similarity:
docId 1  :  0.9998816776422816
docId 2  :  0.9998816776422816
docId 3  :  0.9995442822834324
